In [41]:
import pandas as pd
from prettytable import PrettyTable
import numpy as np
import warnings
from prettytable import PrettyTable
import dtale
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

pd.set_option('display.max_colwidth', None)  # Display full content of each column
pd.set_option('display.max_columns', None)   # Display all columns
pd.set_option('display.width', 5000)         # Set display width



In [42]:
# df=pd.read_csv("Features.csv")
df=pd.read_csv("Merge_Proccessed.csv")
df.shape

(741, 14)

In [14]:
df.columns

Index(['Transaction_Date', 'Description', 'Reference No./Cheque No.', 'Debit', 'Credit', 'Balance', 'Transaction_Type', 'Transaction_Mode', 'DR/CR_Indicator', 'Transaction_ID', 'Recipient_Name', 'Bank', 'UPI_ID', 'Note'], dtype='object')

In [43]:
# dtale.show(df)

In [44]:
# df.isna().sum()
df.describe(include='object').T

,count,unique,top,freq
Transaction_Date,741,360,2024-06-16,14
Description,741,731,CREDIT INTEREST--,8
Reference No./Cheque No.,720,224,TRANSFER TO4897692162094,76
Transaction_Type,741,2,TO,550
Transaction_Mode,731,5,UPI,632
DR/CR_Indicator,741,2,DR,544
Transaction_ID,707,639,XX8237,46
Recipient_Name,707,258,9890160567,56
Bank,630,28,YESB,174
UPI_ID,707,268,paytmqr281,93


Frequency Transaction
--

In [24]:
def categorize_transaction_frequency(filtered_transactions: pd.DataFrame):
    transaction_counts = filtered_transactions['Transaction_Key'].value_counts()
    
    frequency_groups = {
        '1 Transaction': 0, '1-5 Transactions': 0, '5-10 Transactions': 0, 
        '10-35 Transactions': 0, '35-50 Transactions': 0, 'More than 50': 0
    }
    
    for count in transaction_counts:
        if count == 1:
            frequency_groups['1 Transaction'] += 1
        elif 1 < count <= 5:
            frequency_groups['1-5 Transactions'] += 1
        elif 5 < count <= 10:
            frequency_groups['5-10 Transactions'] += 1
        elif 10 < count <= 35:
            frequency_groups['10-35 Transactions'] += 1
        elif 35 < count <= 50:
            frequency_groups['35-50 Transactions'] += 1
        else:
            frequency_groups['More than 50'] += 1
    
    table = PrettyTable()
    table.field_names = ['Category', 'Count']
    for category, count in frequency_groups.items():
        table.add_row([category, count])
    
    return table

def top_10_transactions(filtered_transactions: pd.DataFrame):
    transaction_counts = filtered_transactions['Transaction_Key'].value_counts().head(10)
    
    table = PrettyTable()
    table.field_names = ['Rank', 'Transaction_Key', 'Count']
    
    for i, (key, count) in enumerate(transaction_counts.items(), start=1):
        table.add_row([i, key, count])
    
    return table


Year_Month_Week
--

In [ ]:
def monthly_breakdown(transactions: pd.DataFrame):
    transactions['Transaction_Date'] = pd.to_datetime(transactions['Transaction_Date'])
    transactions['Month'] = transactions['Transaction_Date'].dt.to_period('M')
    monthly_summary = transactions.groupby('Month').agg({'Debit': 'sum', 'Credit': 'sum'}).reset_index()
    
    table = PrettyTable()
    table.field_names = ['Month', 'Total Spent (Debit)', 'Total Income (Credit)']
    for _, row in monthly_summary.iterrows():
        table.add_row([row['Month'], row['Debit'], row['Credit']])
    
    return table

def weekly_breakdown(transactions: pd.DataFrame):
    transactions['Week'] = transactions['Transaction_Date'].dt.to_period('W')
    weekly_summary = transactions.groupby('Week').agg({'Debit': 'sum', 'Credit': 'sum'}).reset_index()
    
    table = PrettyTable()
    table.field_names = ['Week', 'Total Spent (Debit)', 'Total Income (Credit)']
    for _, row in weekly_summary.iterrows():
        table.add_row([row['Week'], row['Debit'], row['Credit']])
    
    return table

def daily_breakdown(transactions: pd.DataFrame):
    daily_summary = transactions.groupby('Transaction_Date').agg({'Debit': 'sum', 'Credit': 'sum'}).reset_index()
    
    table = PrettyTable()
    table.field_names = ['Date', 'Total Spent (Debit)', 'Total Income (Credit)']
    for _, row in daily_summary.iterrows():
        table.add_row([row['Transaction_Date'], row['Debit'], row['Credit']])
    
    return table


Range
--

In [40]:
def transaction_amount_ranges(filtered_transactions: pd.DataFrame):
    transaction_ranges = {
        '0-50': 0, '50-100': 0, '100-500': 0, '500-1000': 0, 
        '1000-5000': 0, '5000-10000': 0, '10000+': 0
    }
    
    for amount in filtered_transactions['Debit'].dropna():
        if amount <= 50:
            transaction_ranges['0-50'] += 1
        elif 50 < amount <= 100:
            transaction_ranges['50-100'] += 1
        elif 100 < amount <= 500:
            transaction_ranges['100-500'] += 1
        elif 500 < amount <= 1000:
            transaction_ranges['500-1000'] += 1
        elif 1000 < amount <= 5000:
            transaction_ranges['1000-5000'] += 1
        elif 5000 < amount <= 10000:
            transaction_ranges['5000-10000'] += 1
        else:
            transaction_ranges['10000+'] += 1
    
    table = PrettyTable()
    table.field_names = ['Transaction Range', 'Count']
    for category, count in transaction_ranges.items():
        table.add_row([category, count])
    
    return table

In [ ]:
# def analyze_top_transactions(transactions: pd.DataFrame, start_date: str, end_date: str):
#     transactions['Transaction_Date'] = pd.to_datetime(transactions['Transaction_Date'])
#     filtered_transactions = transactions[(transactions['Transaction_Date'] >= start_date) &
#                                          (transactions['Transaction_Date'] <= end_date)]
    
#     filtered_transactions['Transaction_Key'] = filtered_transactions['Recipient_Name'].fillna(filtered_transactions['Transaction_ID'])
    
#     print("\nTransaction Frequency Distribution:")
#     print(categorize_transaction_frequency(filtered_transactions))
    
#     print("\nTop 10 Frequent Transactions:")
#     print(top_10_transactions(filtered_transactions))
    
#     print("\nTransaction Amount Distribution:")
#     print(transaction_amount_ranges(filtered_transactions))
    
#     print("\nMonthly Breakdown:")
#     print(monthly_breakdown(filtered_transactions))
    
#     print("\nWeekly Breakdown:")
#     print(weekly_breakdown(filtered_transactions))
    
#     print("\nDaily Breakdown:")
#     print(daily_breakdown(filtered_transactions))
    
# # Example Usage
# analyze_top_transactions(df, '2022-04-01', '2025-12-30')


In [48]:
transactions=df
start_date='2022-04-01' 
end_date='2025-12-30'

transactions['Transaction_Date'] = pd.to_datetime(transactions['Transaction_Date'])
filtered_transactions = transactions[(transactions['Transaction_Date'] >= start_date) &
                                     (transactions['Transaction_Date'] <= end_date)]

filtered_transactions['Transaction_Key'] = filtered_transactions['Recipient_Name'].fillna(filtered_transactions['Transaction_ID'])

print("\nTransaction Frequency Distribution:")
print(categorize_transaction_frequency(filtered_transactions))

print("\nTop 10 Frequent Transactions:")
print(top_10_transactions(filtered_transactions))

print("\nTransaction Amount Distribution:")
print(transaction_amount_ranges(filtered_transactions))

print("\nMonthly Breakdown:")
print(monthly_breakdown(filtered_transactions))

# print("\nWeekly Breakdown:")
# print(weekly_breakdown(filtered_transactions))
    
# print("\nDaily Breakdown:")
# print(daily_breakdown(filtered_transactions))


Transaction Frequency Distribution:
+--------------------+-------+
|      Category      | Count |
+--------------------+-------+
|   1 Transaction    |  156  |
|  1-5 Transactions  |   76  |
| 5-10 Transactions  |   14  |
| 10-35 Transactions |   11  |
| 35-50 Transactions |   0   |
|    More than 50    |   1   |
+--------------------+-------+

Top 10 Frequent Transactions:
+------+-----------------+-------+
| Rank | Transaction_Key | Count |
+------+-----------------+-------+
|  1   |    9890160567   |   56  |
|  2   |     PRACHI S    |   22  |
|  3   |     MrVihaa     |   22  |
|  4   |     IndianR     |   21  |
|  5   |     SUNILSH     |   19  |
|  6   |     JULFIKAR    |   16  |
|  7   |     SNOWCRE     |   15  |
|  8   |     BHARTI A    |   14  |
|  9   |     MumbaiM     |   13  |
|  10  |     SHUBHAM     |   13  |
+------+-----------------+-------+

Transaction Amount Distribution:
+-------------------+-------+
| Transaction Range | Count |
+-------------------+-------+
|       

In [18]:
# def print_selected_columns(df, columns):
#     for index, row in df.iterrows():
#         print(f"Row {index}:")
#         for col in columns:
#             if col in df.columns:
#                 print(f"  {col}: {row[col]}")
#             else:
#                 print(f"  {col}: Column not found!")
#         print("-" * 50)  # Separator for better readability

# # Example usage:
# selected_columns = ["Recipient Name","UPI ID", "Note"]  # Replace with actual column names
# print_selected_columns(df, selected_columns)


In [22]:
def print_transaction_details(df):
    """
    Prints every row with all the extracted details from the DataFrame.
    """
    for index, row in df.iterrows():
        print(f"Row {index}:")
        for col in df.columns:
            print(f"  {col}: {row[col]}")
        print("-" * 50)  # Separator for better readability
print_transaction_details(df[:5])
# print_transaction_details(df)
        


Row 0:
  Transaction Date: 2024-04-02
  Value Date: 2 Apr 2024
  Description: TO TRANSFER-UPI/DR/409378221768/JULFIKAR/YESB/paytmqr1jc/baker-
  Reference No./Cheque No.: TRANSFER TO4897691162095
  Debit: 30.0
  Credit: 0.0
  Balance: 954.4
  Transaction Type: TO
  Transaction Mode: UPI
  DR/CR Indicator: DR
  Transaction ID: 409378221768
  Recipient Name: JULFIKAR
  Bank: YESB
  UPI ID: paytmqr1jc
  Note: baker
--------------------------------------------------
Row 1:
  Transaction Date: 2024-04-04
  Value Date: 4 Apr 2024
  Description: BY TRANSFER-INBIMPS409519772968/9890160567/XX8237/Son-
  Reference No./Cheque No.: MAM000078685023MAM000078685023
  Debit: 0.0
  Credit: 500.0
  Balance: 1454.4
  Transaction Type: BY
  Transaction Mode: INB
  DR/CR Indicator: CR
  Transaction ID: 9890160567
  Recipient Name: XX8237
  Bank: nan
  UPI ID: nan
  Note: Son-
--------------------------------------------------
Row 2:
  Transaction Date: 2024-04-05
  Value Date: 5 Apr 2024
  Description: TO T

  is_weekend: 1
  weekofmonth: 3
  is_month_start: 0
  is_month_end: 0
  is_quarter_start: 0
  is_quarter_end: 0
  hour: 0
  minute: 0
  second: 0
  transaction_amount: 1287.0
  is_large_transaction: 1
  transaction_count_per_day: 3
  transaction_count_per_week: 28
  transaction_count_per_month: 117
  average_transaction_per_day: 685.6666666666666
  cumulative_spent_per_day: 2057.0
  time_since_last_transaction: 0.0
  rolling_avg_transaction_7d: 325.85714285714283
  rolling_avg_transaction_30d: 200.763
  week: 15
  Category: Large Vendor
--------------------------------------------------
Row 167:
  Transaction Date: 2023-04-15 00:00:00
  Value Date: 15 Apr2023
  Description: BY TRANSFER-UPI/CR/310534391690/JitenIs/IDFB/9860566455/Payme-
  Reference No./Cheque No.: TRANSFERFROM4897737162096
  Debit: 0.0
  Credit: 20.0
  Balance: 1309.82
  Transaction Type: BY
  Transaction Mode: UPI
  DR/CR Indicator: CR
  Transaction ID: 310534391690
  Recipient Name: JitenIs
  Bank: IDFB
  UPI ID: 986

  quarter: 4
  month: 11
  year: 2023
  dayofyear: 311
  dayofmonth: 7
  weekofyear: 45
  date_offset: 787
  is_weekend: 0
  weekofmonth: 1
  is_month_start: 0
  is_month_end: 0
  is_quarter_start: 0
  is_quarter_end: 0
  hour: 0
  minute: 0
  second: 0
  transaction_amount: 24.0
  is_large_transaction: 0
  transaction_count_per_day: 3
  transaction_count_per_week: 16
  transaction_count_per_month: 50
  average_transaction_per_day: 96.33333333333331
  cumulative_spent_per_day: 289.0
  time_since_last_transaction: 0.0
  rolling_avg_transaction_7d: 85.71428571428571
  rolling_avg_transaction_30d: 76.76666666666667
  week: 45
  Category: Large Vendor
--------------------------------------------------
Row 403:
  Transaction Date: 2023-11-12 00:00:00
  Value Date: 12 Nov2023
  Description: TO TRANSFER-UPI/DR/331614187502/BHARTI A/AIRP/airtelprep/Payme-
  Reference No./Cheque No.: TRANSFER TO4897696162090
  Debit: 265.0
  Credit: 0.0
  Balance: 1008.92
  Transaction Type: TO
  Transaction Mo

  Value Date: 28 Sep2024
  Description: TO TRANSFER-UPI/DR/427297320117/DILIPKU/YESB/paytmqr5cb/food-
  Reference No./Cheque No.: TRANSFER TO4897696162090
  Debit: 178.0
  Credit: 0.0
  Balance: 336.81
  Transaction Type: TO
  Transaction Mode: UPI
  DR/CR Indicator: DR
  Transaction ID: 427297320117
  Recipient Name: DILIPKU
  Bank: YESB
  UPI ID: paytmqr5cb
  Note: food
  dayofweek: 5
  weekday: Saturday
  quarter: 3
  month: 9
  year: 2024
  dayofyear: 272
  dayofmonth: 28
  weekofyear: 39
  date_offset: 608
  is_weekend: 1
  weekofmonth: 4
  is_month_start: 0
  is_month_end: 0
  is_quarter_start: 0
  is_quarter_end: 0
  hour: 0
  minute: 0
  second: 0
  transaction_amount: 178.0
  is_large_transaction: 0
  transaction_count_per_day: 3
  transaction_count_per_week: 12
  transaction_count_per_month: 52
  average_transaction_per_day: 106.0
  cumulative_spent_per_day: 178.0
  time_since_last_transaction: 0.0
  rolling_avg_transaction_7d: 78.85714285714286
  rolling_avg_transaction_30d:

In [177]:
import holidays
import pandas as pd

indian_holidays = holidays.India(years=2024)

df = pd.DataFrame(indian_holidays.items(), columns=["Date", "Holiday"])
df["Date"] = pd.to_datetime(df["Date"])

# df
